# Pre-preprocessing Sentinel 1

list of `cell_id, date, lat, lon` for downloading

In [4]:
import pandas as pd
import geojson as gsn
from pyproj import Proj
from osgeo import gdal
from osgeo import gdalconst

import tempfile
import wget
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import os
import pickle
from collections import defaultdict
from datetime import datetime, timedelta

import xarray as xr
import rioxarray as rxr

In [2]:
train_feat = pd.read_csv("C:/Users/Matt/Documents/Python Scripts/SnowComp/dat/ground_measures_train_features.csv")
test =pd.read_csv("C:/Users/Matt/Documents/Python Scripts/SnowComp/dat/ground_measures_test_features.csv")
submission = pd.read_csv("C:/Users/Matt/Documents/Python Scripts/SnowComp/dat/submission_format.csv")
train_y = pd.read_csv("C:/Users/Matt/Documents/Python Scripts/SnowComp/dat/train_labels.csv") 
metadata = pd.read_csv("C:/Users/Matt/Documents/Python Scripts/SnowComp/dat/ground_measures_metadata.csv")

In [5]:
path = "C:/Users/Matt/Documents/Python Scripts/SnowComp/dat/grid_cells.geojson"
with open(path) as f:
    gj = gsn.load(f)
print(len(gj['features']))

18130


In [8]:
centroids = {} #cellid : centroid

for cell in range(len(gj['features'])):
    assert len(gj['features'][cell]['geometry']['coordinates'][0]) == 5 #coordinates have repeat on fifth, make sure this is universal
    
    cell_id =gj['features'][cell]['properties']['cell_id']
    centroid = list(np.mean(
        gj['features'][cell]['geometry']['coordinates'][0][0:4],
        axis = 0)) #lazy centroid calculation
    centroids[cell_id] = centroid

## Train labels

In [6]:
#grab date/cell_id combos
dates = {}

#go through all rows, grab nonnull dates
for i in range(train_y.shape[0]):
    dates[train_y.iloc[i,0]] =list(
        train_y.iloc[i,1:][~train_y.iloc[i,:].isnull()].index)#cell_id : [dates]


In [11]:
date_locs = []

# create a numpy array date_locs (cell_id, date, lat, lon)
counter = 0 

for cell, date_list in tqdm(dates.items()):
    for date in date_list:        
        date_locs.append([cell, date, centroids[cell][1], centroids[cell][0]])

        counter += 1
print("total squares:", counter)    
date_locs = pd.DataFrame(date_locs, columns = ['cell_id', 'date', 'lat', 'lon'])

  0%|          | 0/10878 [00:00<?, ?it/s]

total squares: 91490


,cell_id,date,lat,lon
0,0003f387-71c4-48f6-b2b0-d853bd4f0aba,2018-04-26,37.077776,-118.723444
1,0003f387-71c4-48f6-b2b0-d853bd4f0aba,2019-03-16,37.077776,-118.723444
2,0003f387-71c4-48f6-b2b0-d853bd4f0aba,2019-04-18,37.077776,-118.723444
3,0003f387-71c4-48f6-b2b0-d853bd4f0aba,2019-04-28,37.077776,-118.723444
4,0003f387-71c4-48f6-b2b0-d853bd4f0aba,2019-06-08,37.077776,-118.723444
...,...,...,...,...
91485,ffe43514-2c92-43b6-bd84-d183806aca65,2016-03-29,47.904329,-123.502482
91486,fff95195-ccc9-40b7-b302-a0d8570c86bc,2016-02-08,47.735437,-123.376717
91487,fff95195-ccc9-40b7-b302-a0d8570c86bc,2016-03-29,47.735437,-123.376717
91488,fffb4d40-5947-4922-9f05-5d8b5a243d84,2016-02-08,47.523549,-123.798926


In [13]:
date_locs.to_csv("C:/Users/Matt/Dropbox/SnowComp/SentinelHelper/ylabs_dateloc.csv", index= False)